In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from typing import List, Tuple, Optional
from tqdm import tqdm
import matplotlib
from multiagent_misalignment_model import World

In [ ]:
# Set the font to Times New Roman and the output to PDF for the paper submission.
matplotlib.use('pdf')
matplotlib.rcParams['pdf.fonttype'] = 42 # TrueType fonts to avoid Type 3.
matplotlib.rcParams['ps.fonttype'] = 42 # TrueType fonts to avoid Type 3. 
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Times New Roman'] + matplotlib.rcParams['font.serif']

In [ ]:
def plot_multiple_results(results_list,
                          xlabel,
                          ylabel,
                          legend_labels,
                          xlim=None,
                          ylim=(0, 1),
                          xscale='linear',
                          xticks=None,
                          error_bars=False,
                          filename=None,
                          size="small"):
    """
    Plot multiple sets of results on the same graph, with optional error bars.
    
    Args:
        results_list (List[List[Tuple[float, float, float, float]]]): A list of result sets, each containing (x, y_mean, y_min, y_max) tuples.
        xlabel (str): The label for the x-axis.
        ylabel (str): The label for the y-axis.
        legend_labels (List[str]): Labels for each line in the legend.
        xlim (Tuple[float, float], optional): The x-axis limits.
        ylim (Tuple[float, float], optional): The y-axis limits.
        xscale (str, optional): The scale of the x-axis. Defaults to 'linear'.
        xticks (List[int], optional): The x-axis ticks.
        error_bars (bool): Whether to plot error bars or fuzzy/thick lines.
        filename (str, optional): Filename to save the plot as.
        size (str): The size of the plot. Choose from "small" or "regular".
    """
    plt.figure(figsize=(6, 4))  # Adjusted figure size for two-column layout
    
    for results, label in zip(results_list, legend_labels):
        x_values, y_means, y_mins, y_maxs = zip(*results)
        
        if error_bars:
            if size == "small":
                plt.errorbar(x_values, y_means, yerr=[np.array(y_means) - np.array(y_mins), np.array(y_maxs) - np.array(y_means)], 
                             label=label, marker='o', markersize=4, capsize=2, alpha=0.7)
            else:
                plt.errorbar(x_values, y_means, yerr=[np.array(y_means) - np.array(y_mins), np.array(y_maxs) - np.array(y_means)], 
                             label=label, marker='o', markersize=3, capsize=2, alpha=0.7)
        else:
            if size == "small":
                plt.plot(x_values, y_means, label=label, marker='o', markersize=4)
            else:
                plt.plot(x_values, y_means, label=label, marker='o', markersize=3)
            plt.fill_between(x_values, y_mins, y_maxs, alpha=0.2)
    
    if size == "small":
        plt.xlabel(xlabel, fontsize=12)
        plt.ylabel(ylabel, fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(fontsize=10, loc='best')
    else:
        plt.xlabel(xlabel, fontsize=10)
        plt.ylabel(ylabel, fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(fontsize=8, loc='best')

    if xlim is not None:
        plt.xlim(xlim)
    if ylim is not None:
        plt.ylim(ylim)

    plt.xscale(xscale)

    if xscale == 'log' or (xscale == 'linear' and xticks is not None):
        plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
        if xscale == 'log':
            plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
        plt.gca().xaxis.set_minor_formatter(ticker.NullFormatter())
        plt.gca().xaxis.set_minor_locator(ticker.NullLocator())
        plt.gca().xaxis.set_major_locator(ticker.FixedLocator(xticks))
    elif xscale == 'linear':
        plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

    plt.tick_params(axis='both', which='major', labelsize=10)
    plt.tight_layout()

    if filename:
        plt.savefig(filename + ".pdf", format='pdf', bbox_inches='tight')
    else:
        print("No filename provided. Plot will be saved with default name \"plot.pdf\".")
        plt.savefig("plot.pdf", format='pdf', bbox_inches='tight')

In [ ]:
def run_varying_problem_areas_experiment(min_agents: int = 1,
                          max_agents: int = 100,
                          num_steps: int = 15,
                          num_goals: int = 3,
                          problem_area_counts: List[int] = [1, 2, 3, 4, 5],
                          single_conflict_value: Optional[float] = 0.5,
                          randomize_conflict: bool = False,
                          random_conflict_range: Tuple[float, float] = (0.1, 0.9),
                          random_goals: bool = False,
                          round_robin_goals: bool = True,
                          random_weight_range: Tuple[float, float] = (0, 1),
                          num_runs: int = 10,
                          error_bars: bool = False,
                          normalize_by_goals: bool = False,
                          filename: str = "") -> None:
    """
    Run the experiment with different numbers of problem areas and plot the results. Repeat the experiment multiple times to get statistical data.

    Args:
        min_agents (int): The minimum number of agents to start with.
        max_agents (int): The maximum number of agents to simulate.
        num_steps (int): The number of logarithmic steps to take between min and max agents.
        num_goals (int): The number of goals per problem area.
        problem_area_counts (List[int]): The number of problem areas to create.
        single_conflict_value (Optional[float]): Single conflict value to use for all goal pairs. If provided, this will create or override the conflict matrix.
        randomize_conflict (bool): Whether to randomly generate conflict values for this problem area.
        random_conflict_range (Tuple[float, float]): Range of conflict values (min, max). Must be in range [0, 1].
        random_goals (bool): Whether to assign random goals to the agents.
        num_runs (int): Number of times to run each experiment to get statistical data.
        error_bars (bool): Whether to plot error bars or fuzzy/thick lines.
        normalize_by_goals (bool): Whether to normalize the misalignment by the number of goals.
        filename (str): Filename to save the plot as.
    """
    print("\nExperiment: Changing the number of agents and problem areas")
    
    results_list = []
    legend_labels = []
    
    # Generate logarithmically spaced number of agents
    agent_counts = np.logspace(np.log10(min_agents), np.log10(max_agents), num=num_steps, dtype=int)
    agent_counts = np.unique(agent_counts)  # Remove any duplicates due to integer conversion

    for num_problem_areas in problem_area_counts:
        problem_area_results = []
        
        for num_agents in tqdm(agent_counts, desc=f"Agents (PAs: {num_problem_areas})"):
            run_results = []
            
            for _ in range(num_runs):
                world = World()
                
                # Add problem areas
                for _ in range(num_problem_areas):
                    if randomize_conflict:
                        world.add_problem_area(num_goals, randomize_conflict=randomize_conflict, random_conflict_range=random_conflict_range)
                    else:
                        world.add_problem_area(num_goals, single_conflict_value=single_conflict_value)
                
                world.add_agents_in_bulk(num_agents, random_goals=random_goals, random_weight_range=random_weight_range, round_robin_goals=round_robin_goals)
                
                misalignment = world.calculate_population_misalignment(normalize_by_goal_number=normalize_by_goals)

                run_results.append(misalignment)
            
            mean_misalignment = np.mean(run_results)
            min_misalignment = np.min(run_results)
            max_misalignment = np.max(run_results)

            problem_area_results.append((num_agents, mean_misalignment, min_misalignment, max_misalignment))
        
        results_list.append(problem_area_results)
        if num_problem_areas == 1:
            legend_labels.append(f"{num_problem_areas} Problem Area")
        else:
            legend_labels.append(f"{num_problem_areas} Problem Areas")

    # Plot the results
    plot_multiple_results(
        results_list,
        "Number of Agents",
        "Population Misalignment",
        legend_labels,
        xlim=(min_agents, max_agents),
        ylim=(0,1),
        xscale='log',
        xticks=agent_counts,
        error_bars=error_bars,
        filename=filename
    )

    print(f"Population misalignment with max agents ({max_agents}): {results_list[-1][-1][1]}")

In [ ]:
run_varying_problem_areas_experiment(
    min_agents=1,
    max_agents=100,
    num_steps=20,
    num_goals=3,
    problem_area_counts=[1, 2, 3, 4, 5],
    randomize_conflict=False,
    random_goals=True,
    round_robin_goals=False,
    single_conflict_value=1,
    #random_conflict_range=(0.1, 0.9),
    random_weight_range=(1, 1),
    num_runs=100,
    error_bars=False,
    filename="x_axis_agents_lines_problem_areas"
)

In [ ]:
def run_varying_goals_experiment(min_agents: int = 1,
                          max_agents: int = 100,
                          num_steps: int = 15,
                          goal_counts: List[int] = [2, 3, 4, 5],
                          num_problem_areas: int = 3,
                          single_conflict_value: Optional[float] = 0.5,
                          randomize_conflict: bool = False,
                          random_conflict_range: Tuple[float, float] = (0.1, 0.9),
                          random_goals: bool = False,
                          round_robin_goals: bool = True,
                          random_weight_range: Tuple[float, float] = (0, 1),
                          num_runs: int = 10,
                          error_bars: bool = False,
                          normalize_by_goals: bool = False,
                          filename: str = "") -> None:
    """
    Run the experiment with different numbers of goals and plot the results.

    Args:
        min_agents (int): The minimum number of agents to start with.
        max_agents (int): The maximum number of agents to simulate.
        num_steps (int): The number of logarithmic steps to take between min and max agents.
        goal_counts (List[int]): The number of goals per problem area to test.
        num_problem_areas (int): The number of problem areas to create.
        single_conflict_value (Optional[float]): Single conflict value to use for all goal pairs. If provided, this will create or override the conflict matrix.
        randomize_conflict (bool): Whether to randomly generate conflict values for this problem area.
        random_conflict_range (Tuple[float, float]): Range of conflict values (min, max). Must be in range [0, 1].
        random_goals (bool): Whether to assign random goals to the agents.
        round_robin_goals (bool): Whether to assign goals in a round-robin fashion.
        random_weight_range (Tuple[float, float]): The range of weights for the agents if goals/weights are being assigned randomly.
        num_runs (int): Number of times to run each experiment to get statistical data.
        error_bars (bool): Whether to plot error bars or fuzzy/thick lines.
        normalize_by_goals (bool): Whether to normalize the population misalignment by the number of goals in the considered pa.
        filename (str): Filename to save the plot as.
    """
    print("\nExperiment: Changing the number of agents and goals")
    
    results_list = []
    legend_labels = []
    
    # Generate logarithmically spaced number of agents
    agent_counts = np.logspace(np.log10(min_agents), np.log10(max_agents), num=num_steps, dtype=int)
    agent_counts = np.unique(agent_counts)  # Remove any duplicates due to integer conversion

    for num_goals in goal_counts:
        goal_results = []
        
        for num_agents in tqdm(agent_counts, desc=f"Agents (Goals: {num_goals})"):
            run_results = []
            
            for _ in range(num_runs):
                world = World()

                # Add problem areas
                for _ in range(num_problem_areas):
                    if randomize_conflict:
                        world.add_problem_area(num_goals, randomize_conflict=randomize_conflict, random_conflict_range=random_conflict_range)
                    else:
                        world.add_problem_area(num_goals, single_conflict_value=single_conflict_value)

                world.add_agents_in_bulk(num_agents, random_goals=random_goals, round_robin_goals=round_robin_goals, random_weight_range=random_weight_range)
                
                misalignment = world.calculate_population_misalignment(normalize_by_goal_number=normalize_by_goals)
                run_results.append(misalignment)
            
            mean_misalignment = np.mean(run_results)
            min_misalignment = np.min(run_results)
            max_misalignment = np.max(run_results)
            
            goal_results.append((num_agents, mean_misalignment, min_misalignment, max_misalignment))
        
        results_list.append(goal_results)
        legend_labels.append(f"{num_goals} Goals")

    # Plot the results
    plot_multiple_results(
        results_list,
        "Number of Agents",
        "Population Misalignment",
        legend_labels,
        xlim=(min_agents, max_agents),
        ylim=(0,1),
        xscale='log',
        xticks=agent_counts,
        error_bars=error_bars,
        filename=filename
    )

In [ ]:
run_varying_goals_experiment(
    min_agents=1,
    max_agents=100,
    num_steps=20,
    goal_counts=[2, 3, 4, 5],
    num_problem_areas=4,
    single_conflict_value=1,
    randomize_conflict=False,
    random_conflict_range=(0.1, 0.9),
    random_goals=True,
    round_robin_goals=False,
    random_weight_range=(1, 1),
    num_runs=100,
    error_bars=False,  # or False for fuzzy/thick lines
    normalize_by_goals=False,
    filename="x_axis_agents_lines_goals"
)

In [ ]:
def run_weight_sensitivity_experiment(
    num_agents: int = 100,
    problem_area_counts: List[int] = [1, 2, 3, 4],
    goal_counts: List[int] = [2, 4],
    target_goal: int = 1,
    weight_range: np.ndarray = np.linspace(0, 1, 100),
    single_conflict_value: Optional[float] = 0.5,
    randomize_conflict: bool = False,
    random_conflict_range: Tuple[float, float] = (0.1, 0.9),
    random_goals: bool = False,
    round_robin_goals: bool = True,
    random_weight_range: Tuple[float, float] = (0, 1),
    allow_no_goal: bool = False,
    normalize_weights: bool = False,
    num_runs: int = 10,
    error_bars: bool = False,
    filename: str = "",
    pa_misalignment_averaging_style: str = "geometric mean"
)-> None:
    """
    Run the experiment with different numbers of problem areas and plot the results.

    Args:
        num_agents (int): The number of agents to simulate.
        problem_area_counts (List[int]): The number of problem areas to create.
        goal_counts (List[int]): The number of goals per problem area to test.
        target_goal (int): The target goal to change the weight for.
        weight_range (np.ndarray): The range of weights to test.
        single_conflict_value (Optional[float]): Single conflict value to use for all goal pairs.
        randomize_conflict (bool): Whether to randomly generate conflict values for this problem area.
        random_conflict_range (Tuple[float, float]): Range of conflict values (min, max).
        random_goals (bool): Whether to assign random goals to the agents.
        round_robin_goals (bool): Whether to assign goals in a round-robin fashion.
        random_weight_range (Tuple[float, float]): The range of weights for randomly assigned goals.
        allow_no_goal (bool): Whether to allow agents to have no goal.
        normalize_weights (bool): Whether to normalize the weights after updating the target group.
        num_runs (int): Number of times to run each experiment to get statistical data.
        error_bars (bool): Whether to plot error bars or fuzzy/thick lines.
        filename (str): Filename to save the plot as.
        pa_misalignment_averaging_style (str): The style of averaging to use when calculating the population misalignment. Choose one of "arithmetic mean", "geometric mean", "harmonic mean", "min", "max", or "disregard weights".
    """
    print("\nExperiment: Changing agents' weights for particular goals with varying problem areas")    
    results_list = []
    legend_labels = []

    for num_goals in goal_counts:
        for pa_count in problem_area_counts:    
            print(f"\nRunning simulation with {num_goals} goals and {pa_count} problem areas...")
            pa_results = []
            
            for weight in tqdm(weight_range, desc="Weights"):
                run_results = []
                
                for _ in range(num_runs):
                    world = World()
                    
                    # Add problem areas
                    for _ in range(pa_count):
                        if randomize_conflict:
                            world.add_problem_area(num_goals, randomize_conflict=randomize_conflict, random_conflict_range=random_conflict_range)
                        else:
                            world.add_problem_area(num_goals, single_conflict_value=single_conflict_value)
                    
                    # Add agents
                    world.add_agents_in_bulk(num_agents, random_goals=random_goals, round_robin_goals=round_robin_goals, 
                                            random_weight_range=random_weight_range, allow_no_goal=allow_no_goal)
                    
                    # Update weights for the target goal
                    for agent in world.agents:
                        if agent.goals[0] == target_goal:
                            new_goals = {0: (target_goal, weight)}
                            world.update_agent(agent.id, new_goals, normalize_weights=normalize_weights)
                    
                    misalignment = world.calculate_population_misalignment(problem_area_id=None, normalize_by_goal_number=False, pa_misalignment_averaging_style=pa_misalignment_averaging_style)
                    run_results.append(misalignment)
                
                mean_misalignment = np.mean(run_results)
                min_misalignment = np.min(run_results)
                max_misalignment = np.max(run_results)
                
                pa_results.append((weight, mean_misalignment, min_misalignment, max_misalignment))
            
            results_list.append(pa_results)
            if pa_count == 1:
                legend_labels.append(f"{num_goals} Goals each, {pa_count} Problem Area")
            else:
                legend_labels.append(f"{num_goals} Goals each, {pa_count} Problem Areas")

    xticks = np.linspace(0, 1, 11)

    plot_multiple_results(
        results_list,
        "Goal Weight",
        "Population Misalignment",
        legend_labels,
        xlim=(0, 1),
        xticks=xticks,
        error_bars=error_bars,
        filename=filename,
        size="regular"
    )

In [ ]:
run_weight_sensitivity_experiment(
    num_agents=100,
    problem_area_counts=[1, 2, 3, 4],
    goal_counts=[2, 4],
    target_goal=1,
    weight_range=np.linspace(0, 1, 50),
    single_conflict_value=1,
    randomize_conflict=False,
    random_conflict_range=(0.1, 0.9),
    random_goals=True,
    round_robin_goals=False,
    random_weight_range=(1, 1),
    allow_no_goal=False,
    normalize_weights=False,
    num_runs=100,
    error_bars=False,
    filename="x_axis_goal_weight_lines_problem_areas",
    pa_misalignment_averaging_style="arithmetic mean"
)

In [ ]:
def calculate_goal_distribution(primary_proportion: float, num_goals: int, method: str = 'uniform') -> List[float]:
    """
    Calculate the distribution of agents across goals.

    Args:
        primary_proportion (float): The proportion of agents assigned to the primary goal.
        num_goals (int): The total number of goals.
        method (str): The method to distribute remaining agents ('uniform' or 'halving').

    Returns:
        List[float]: A list of proportions for each goal.
    """
    if primary_proportion < 0 or primary_proportion > 1:
        raise ValueError("Primary proportion must be between 0 and 1.")

    if method not in ['uniform', 'halving']:
        raise ValueError("Distribution method must be 'uniform' or 'halving'.")

    remaining_proportion = 1 - primary_proportion
    distribution = [primary_proportion]

    if method == 'uniform':
        other_proportions = [remaining_proportion / (num_goals - 1)] * (num_goals - 1)
    else:  # halving method
        other_proportions = []
        for _ in range(num_goals - 2):
            prop = remaining_proportion / 2
            other_proportions.append(prop)
            remaining_proportion -= prop
        other_proportions.append(remaining_proportion)

    distribution.extend(other_proportions)
    return distribution

In [ ]:
def run_goal_distribution_experiment(
    num_agents: int = 100,
    num_problem_areas: int = 1,
    max_num_goals: int = 4,
    distribution_method: Optional[str] = None,
    proportion_range: np.ndarray = np.linspace(0, 1, 101),
    single_conflict_value: Optional[float] = 0.5,
    randomize_conflict: bool = False,
    random_conflict_range: Tuple[float, float] = (0.1, 0.9),
    random_weight_range: Tuple[float, float] = (1, 1),
    normalize_weights: bool = True,
    num_runs: int = 10,
    error_bars: bool = False,
    filename: str = ""
) -> Tuple[List[Tuple[float, float, float, float]], str]:
    """
    Run an experiment to show how alignment changes as agents are redistributed from one goal to others.
    Assumes that all agents will have a non-zero goal.

    Args:
        num_agents (int): The number of agents to simulate.
        num_problem_areas (int): The number of problem areas to create.
        max_num_goals (int): The maximum number of goals to test.
        distribution_method (str): Method to distribute remaining agents ('uniform' or 'halving'). If none, do both.
        proportion_range (np.ndarray): The range of proportions for the primary goal to test.
        single_conflict_value (Optional[float]): Single conflict value to use for all goal pairs.
        randomize_conflict (bool): Whether to randomly generate conflict values for this problem area.
        random_conflict_range (Tuple[float, float]): Range of conflict values (min, max).
        random_weight_range (Tuple[float, float]): The range of weights for randomly assigned goals.
        normalize_weights (bool): Whether to normalize the weights after updating the target group.
        num_runs (int): Number of times to run each experiment to get statistical data.
        error_bars (bool): Whether to plot error bars or fuzzy/thick lines.
        filename (str): Filename to save the plot as.

    Returns:
        Tuple[List[Tuple[float, float, float, float]], str]: A list of results, where each result is a tuple of 
        (proportion, mean_misalignment, min_misalignment, max_misalignment), and a legend label.
    """
    print("\nExperiment: Changing distribution of agents across goals")
    
    results_list = []
    legend_labels = []

    for num_goals in range(2, max_num_goals + 1):

        if distribution_method is None:
            if num_goals == 2:
                distribution_methods = ['uniform'] # With only 2 goals, halving method is the same as uniform
            else:
                distribution_methods = ['uniform', 'halving']
        else:
            distribution_methods = [distribution_method]

        for method in distribution_methods:
            
            print(f"\nRunning simulation with {num_goals} goals and {method} distribution...")
            results = []

            for proportion in tqdm(proportion_range, desc="Proportions"):
                
                run_results = []
                for _ in range(num_runs):
                    world = World()
                    
                    # Add problem areas
                    for _ in range(num_problem_areas):
                        if randomize_conflict:
                            world.add_problem_area(num_goals, randomize_conflict=randomize_conflict, random_conflict_range=random_conflict_range)
                        else:
                            world.add_problem_area(num_goals, single_conflict_value=single_conflict_value)

                    goal_distribution = calculate_goal_distribution(proportion, num_goals, method)
                    
                    # Add agents with specified goal distribution
                    for goal, goal_proportion in enumerate(goal_distribution):
                        goal = goal + 1  # Don't assign agents to the zero goal; start from 1 instead
                        num_agents_for_goal = int(num_agents * goal_proportion)
                        world.add_agents_in_bulk(
                            num_agents_for_goal,
                            specified_goals={pa: (goal, np.random.uniform(*random_weight_range)) for pa in range(num_problem_areas)},
                            normalize_weights=normalize_weights
                        )
                    
                    misalignment = world.calculate_population_misalignment()
                    run_results.append(misalignment)
                
                mean_misalignment = np.mean(run_results)
                min_misalignment = np.min(run_results)
                max_misalignment = np.max(run_results)
                
                results.append((proportion, mean_misalignment, min_misalignment, max_misalignment))
            
            results_list.append(results)
            legend_labels.append(f"{num_goals} Goals, {method.capitalize()} Distribution")
    
    xticks = np.linspace(0, 1, 11)

    plot_multiple_results(
        results_list,
        "Proportion of Agents with Goal 1",
        "Population Misalignment",
        legend_labels,
        xlim=(0, 1),
        xticks=xticks,
        error_bars=error_bars,
        filename=filename
    )

In [ ]:
num_agents=1000
num_problem_areas=1
max_num_goals=5
proportion_range=np.linspace(0, 1, 51)
single_conflict_value=1
randomize_conflict=False
random_conflict_range=(0.1, 0.9)
random_weight_range=(1, 1)
normalize_weights=False
num_runs=1
error_bars=False
filename="x_axis_proportion_lines_goal_distributions"

distribution_method = 'uniform'
run_goal_distribution_experiment(num_agents, num_problem_areas, max_num_goals, distribution_method, proportion_range, single_conflict_value, randomize_conflict, random_conflict_range, random_weight_range, normalize_weights, num_runs, error_bars, filename+"_uniform")

In [ ]:
def run_car_pedestrian_experiment_with_different_weights(
    total_agents: int = 10,
    primary_car_weights: List[float] = [0.50, 0.40, 0.35, 0.30, 0.20, 0.30, 0.30, 0.30],
    primary_ped_weights: List[float] = [0.99, 0.15, 0.15, 0.05, 0.05, 0.05, 0.01, 0.05],
    primary_weight_title: str = "Scenario",
    secondary_car_weights: List[float] = [0.45, 0.35, 0.30, 0.25, 0.15, 0.25, 0.25, 0.25],
    secondary_ped_weights: List[float] = [0.95, 0.10, 0.10, 0.03, 0.03, 0.03, 0.01, 0.03],
    secondary_weight_title: str = "Maximum",
    proportion_range: np.ndarray = np.linspace(0, 1, 101),
    conflict_values: List[float] = [0.2, 0.5, 0.8],
    normalize_weights: bool = False,
    num_runs: int = 10,
    error_bars: bool = False,
    filename: str = ""
) -> None:
    """
    Run an experiment to compare "car" agents to "pedestrian" agents with varying proportions, conflict matrices,
    and primary/secondary weight sets.

    Args:
        total_agents (int): The total number of agents to simulate.
        primary_car_weights (List[float]): The primary weights for car agents' goals in each problem area.
        primary_ped_weights (List[float]): The primary weights for pedestrian agents' goals in each problem area.
        primary_weight_title (str): The title for the primary weight set.
        secondary_car_weights (List[float]): The secondary weights for car agents' goals in each problem area.
        secondary_ped_weights (List[float]): The secondary weights for pedestrian agents' goals in each problem area.
        secondary_weight_title (str): The title for the secondary weight set.
        proportion_range (np.ndarray): The range of proportions of car agents to test.
        conflict_values (List[float]): The conflict values to test between car and pedestrian goals.
        normalize_weights (bool): Whether to normalize the weights after setting goals.
        num_runs (int): Number of times to run each experiment to get statistical data.
        error_bars (bool): Whether to plot error bars or fuzzy/thick lines.
        filename (str): Filename to save the plot as.
    """
    print("\nExperiment: Comparing car agents to pedestrian agents with varying proportions, conflict matrices, and weight sets")
    
    results_list = []
    legend_labels = []

    weight_sets = [
        (primary_weight_title, primary_car_weights, primary_ped_weights),
        (secondary_weight_title, secondary_car_weights, secondary_ped_weights)
    ]

    if len(primary_car_weights) != len(primary_ped_weights) or \
       len(secondary_car_weights) != len(secondary_ped_weights) or \
       len(primary_car_weights) != len(secondary_car_weights):
        raise ValueError("All weight lists must have the same length.")

    num_problem_areas = len(primary_car_weights)

    for weight_set_name, car_weights, ped_weights in weight_sets:
        
        for conflict_value in conflict_values:
            conflict_matrix = np.array([[0, conflict_value], [conflict_value, 0]])
        
            print(f"\nRunning simulation with {weight_set_name} weights and conflict value {conflict_value}...")

            results = []

            for car_proportion in tqdm(proportion_range, desc="Car Proportions"):
                run_results = []
                
                for _ in range(num_runs):
                    world = World()
                    
                    # Add problem areas
                    for _ in range(num_problem_areas):
                        world.add_problem_area(num_goals=2, conflict_matrix=conflict_matrix)
                    
                    # Add car agents
                    num_car_agents = int(total_agents * car_proportion)
                    car_goals_dict = {pa: (1, weight) for pa, weight in enumerate(car_weights)}
                    world.add_agents_in_bulk(num_car_agents, specified_goals=car_goals_dict, normalize_weights=normalize_weights)
                    
                    # Add pedestrian agents
                    num_ped_agents = total_agents - num_car_agents
                    ped_goals_dict = {pa: (2, weight) for pa, weight in enumerate(ped_weights)}
                    world.add_agents_in_bulk(num_ped_agents, specified_goals=ped_goals_dict, normalize_weights=normalize_weights)
                    
                    misalignment = world.calculate_population_misalignment()
                    run_results.append(misalignment)
                
                mean_misalignment = np.mean(run_results)
                min_misalignment = np.min(run_results)
                max_misalignment = np.max(run_results)
                
                results.append((car_proportion, mean_misalignment, min_misalignment, max_misalignment))
            
            results_list.append(results)
            legend_labels.append(f"{weight_set_name} weights, conflict value: {conflict_value}")
    
    xticks = np.linspace(0, 1, 11)

    plot_multiple_results(
        results_list,
        "Proportion of Agents Which are Autonomous Vehicles",
        "Population Misalignment",
        legend_labels,
        xlim=(0, 1),
        xticks=xticks,
        error_bars=error_bars,
        filename=filename
    )

In [ ]:
total_agents=1000
primary_car_weights = [0.50, 0.40, 0.35, 0.30, 0.20, 0.30, 0.30, 0.30]
primary_ped_weights = [0.99, 0.15, 0.15, 0.05, 0.05, 0.05, 0.01, 0.05]
primary_weight_title="Scenario"
secondary_car_weights=[1, 1, 1, 1, 1, 1, 1, 1]
secondary_ped_weights=[1, 1, 1, 1, 1, 1, 1, 1]
secondary_weight_title="Maximum"
proportion_range=np.linspace(0, 1, 51)
conflict_values=[0.5, 1]
normalize_weights=False
num_runs=1
error_bars=False
filename="x_axis_car_pedestrian_lines_conflict_and_weights"

run_car_pedestrian_experiment_with_different_weights(total_agents, primary_car_weights, primary_ped_weights, primary_weight_title, secondary_car_weights, secondary_ped_weights, secondary_weight_title, proportion_range, conflict_values, normalize_weights, num_runs, error_bars, filename)

In [ ]:

def run_conflict_levels_experiment(
    num_agents: int = 120,
    min_goals: int = 2,
    max_goals: int = 10,
    conflict_lists: List[List[float]] = [[0.17, 0.33, 0.5], [0.33, 0.5, 0.67], [0.5, 0.67, 0.83]],
    weight_range: Tuple[float, float] = (0.1, 1.0),
    num_runs: int = 10,
    error_bars: bool = False,
    normalize_by_goals: bool = False,
    filename: str = ""
) -> None:
    """
    Run an experiment to compare different conflict levels with varying numbers of goals.

    Args:
        num_agents (int): The constant number of agents to simulate.
        min_goals (int): The minimum number of goals per problem area.
        max_goals (int): The maximum number of goals per problem area.
        conflict_lists (List[List[float]]): A list of lists, where each inner list contains
                                            the conflict levels for each problem area.
        weight_range (Tuple[float, float]): The range of weights for goals (min, max).
        num_runs (int): Number of times to run each experiment to get statistical data.
        error_bars (bool): Whether to plot error bars or fuzzy/thick lines.
        normalize_by_goals (bool): Whether to normalize the misalignment by the number of goals.
        filename (str): Filename to save the plot as.
    """
    print("\nExperiment: Changing the number of goals with different conflict levels")

    goal_counts = range(min_goals, max_goals + 1)

    results_list = []
    legend_labels = []

    for conflict_set in conflict_lists:
        num_problem_areas = len(conflict_set)
        problem_area_results = []
        
        for num_goals in tqdm(goal_counts, desc=f"Goals (PAs: {num_problem_areas}, Normalized: {normalize_weights})"):
            run_results = []
            
            for _ in range(num_runs):
                world = World()
                
                for conflict_value in conflict_set:
                    world.add_problem_area(num_goals, single_conflict_value=conflict_value)
                
                agents_per_goal = num_agents // num_goals
                remaining_agents = num_agents % num_goals
                
                for goal in range(1, num_goals + 1):
                    goal_agents = agents_per_goal + (1 if goal <= remaining_agents else 0)
                    world.add_agents_in_bulk(
                        goal_agents,
                        specified_goals={pa: (goal, np.random.uniform(*weight_range)) for pa in range(num_problem_areas)},
                        normalize_weights=normalize_weights
                    )
                
                misalignment = world.calculate_population_misalignment(normalize_by_goal_number=normalize_by_goals)
                run_results.append(misalignment)
            
            mean_misalignment = np.mean(run_results)
            min_misalignment = np.min(run_results)
            max_misalignment = np.max(run_results)

            problem_area_results.append((num_goals, mean_misalignment, min_misalignment, max_misalignment))
        
        results_list.append(problem_area_results)
        legend_labels.append(f"{num_problem_areas} PA{"s" if num_problem_areas != 1 else ""}, Conflict levels: {', '.join([f'{c:.2f}' for c in conflict_set])}")
    

    plot_multiple_results(
        results_list,
        "Number of Goals per Problem Area",
        "Population Misalignment",
        legend_labels,
        xlim=(min_goals, max_goals),
        ylim=(0,1),
        xscale='linear',
        error_bars=error_bars,
        filename=filename,
        size="small"
    )

    print("Experiment completed. Results saved.")

In [ ]:
num_agents=120
min_goals=1
max_goals=10
normalize_weights=False
weight_range=(0.25, 0.75)
num_runs=100
error_bars=False
normalize_by_goals=False
filename="x_axis_goals_lines_conflict_lists"

conflict_lists = [
    [1.0, 1.0, 1.0],
    [1.0],
    [1.0, 0.75],
    [1.0, 0.75, 0.5],
    [0.5],
    [0.25, 0.5, 0.75],
    [0.25, 0.5],
    [0.25],
    [0.25, 0.25, 0.25]
]

run_conflict_levels_experiment(num_agents, min_goals, max_goals, conflict_lists, weight_range, num_runs, error_bars, normalize_by_goals, filename)

In [ ]:
shopping_test_world = World()
# PA 0: p_f. Goals g^c, g^R, g^{R^S}.
shopping_test_world.add_problem_area(3, conflict_matrix=np.array([[0, 0.1, 0.1], [0.1, 0, 0.1], [0.1, 0.1, 0]]))  
# PA 1: p_h. Goals g^c, g^R, g^{R^S}. 
shopping_test_world.add_problem_area(3, conflict_matrix=np.array([[0, 0.5, 0.9], [0.5, 0, 0.3], [0.9, 0.1, 0.3]]))
shopping_test_world.add_agent(goals={0: (1, 0.8), 1: (1, 0.6)}) # c
shopping_test_world.add_agent(goals={0: (2, 0.9), 1: (2, 0.7)}) # R
shopping_test_world.add_agent(goals={0: (3, 1.0), 1: (3, 1.0)}) # R^S

print(f"Misalignment for p_f: {shopping_test_world.calculate_population_misalignment(0)}")
print(f"Misalignment for p_h: {shopping_test_world.calculate_population_misalignment(1)}")

print("Overall Population Misalignment: ", shopping_test_world.calculate_population_misalignment())